In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm, naive_bayes
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
import seaborn as sns
import pickle

read data

In [ ]:
data = pd.read_csv('../dataset/news_politics_final.csv')

split data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data["Content_processed"], data["label"], test_size= 0.3, random_state= 42)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

extract feature

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vec = TfidfVectorizer(min_df=2)
tf_idf_train = tf_idf_vec.fit_transform(X_train)
tf_idf_train

turning parameters

In [ ]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
# Set up score
scoring = ['accuracy']
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
# Define grid search
grid_search = GridSearchCV(estimator=svm.SVC(), 
                           param_grid=param_grid, 
                           scoring=scoring, 
                           refit='accuracy', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)
# Fit grid search
grid_result = grid_search.fit(X_train, Y_train)
# Print grid search summary
grid_result.best_params_

modeling

classifier = svm.SVC(C=10, gamma=0.1, kernel="sigmoid")
classifier.fit(tf_idf_train, Y_train)
X_test_tf_idf = tf_idf_vec.transform(X_test)
y_pred_tf_idf_svm = classifier.predict(X_test_tf_idf)
heatconmat(y_pred_tf_idf_svm, Y_test)
print(classification_report(y_pred_tf_idf_svm, Y_test, digits=4))